In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [ ]:
# Setup
fake = Faker()
np.random.seed(42)
random.seed(42)

# Parameters
num_customers = 19440
num_days = 1800
start_date = datetime(2024, 10, 1)

# U.S. states (regions) with unequal sales distribution weights
regions = {
    "California": 1.3,
    "Texas": 1.2,
    "New York": 1.0,
    "Florida": 1.1,
    "Wisconsin": 0.8,
    "Colorado": 0.6,
    "Alabama": 0.5
}

# Payment methods
payment_methods = ["Cash", "Credit Card", "Debit Card", "Mobile App"]

# Product categories and product names
categories = {
    "Beverages": ["Apple Juice", "Orange Soda", "Iced Tea", "Mineral Water"],
    "Bakery": ["Bread Loaf", "Croissant", "Muffin", "Bagel"],
    "Deli": ["Chicken Wrap", "Ham Sandwich", "Turkey Roll", "Veggie Wrap"],
    "Snacks": ["Potato Chips", "Granola Bar", "Chocolate Cookie", "Popcorn"],
    "Dairy": ["Milk", "Cheese Block", "Yogurt", "Butter"],
    "Frozen Foods": ["Frozen Pizza", "Ice Cream", "Frozen Vegetables", "Frozen Fries"],
    "Produce": ["Banana", "Apple", "Carrot", "Lettuce"],
    "Meat": ["Chicken Breast", "Ground Beef", "Pork Chops", "Bacon"],
    "Seafood": ["Salmon Fillet", "Shrimp Pack", "Tuna Can", "Crab Meat"],
    "Pantry": ["Pasta", "Rice", "Canned Beans", "Tomato Sauce"]
}

# Generate product dataset
products = []
product_id = 1
for category, items in categories.items():
    for item in items:
        price = round(np.random.uniform(1.0, 5.0), 2)
        products.append({
            "product_id": product_id,
            "name": item,
            "category": category,
            "price": price
        })
        product_id += 1
df_products = pd.DataFrame(products)

# Generate customer dataset
customers = []
for i in range(1, num_customers + 1):
    region = random.choices(list(regions.keys()), weights=list(regions.values()), k=1)[0]
    customers.append({
        "customer_id": i,
        "name": fake.name(),
        "region": region,
        "customer_type": random.choices(["Regular", "New", "Loyal"], weights=[0.5, 0.3, 0.2])[0]
    })
df_customers = pd.DataFrame(customers)

# Generate sales dataset
sales = []
sale_id = 1
store_id = 1

for day in range(num_days):
    date = start_date + timedelta(days=day)
    for region, weight in regions.items():
        growth_factor = 1 + (day / num_days) * 0.4  # simulate gradual growth
        base_sales = int(np.random.poisson(lam=10 * weight) * growth_factor)

        # Boost weekend sales
        if date.weekday() >= 5:
            base_sales += np.random.randint(5, 15)

        for _ in range(base_sales):
            product = df_products.sample(1).iloc[0]
            customer = df_customers[df_customers["region"] == region].sample(1).iloc[0]
            quantity = np.random.randint(1, 4)

            # Regional and customer-based quantity adjustments
            if region == "Wisconsin" and product["category"] == "Dairy":
                quantity += 1
            if region == "Florida" and product["category"] == "Beverages":
                quantity += 1
            if customer["customer_type"] == "Loyal":
                quantity += 1

            # Payment method probabilities based on customer type
            payment_method = random.choices(
                payment_methods,
                weights=[0.2, 0.4, 0.2, 0.2] if customer["customer_type"] == "Loyal" else [0.4, 0.3, 0.2, 0.1],
                k=1
            )[0]

            sales.append({
                "sale_id": sale_id,
                "product_id": product["product_id"],
                "customer_id": customer["customer_id"],
                "store_id": store_id,
                "region": region,
                "date": date,
                "quantity": quantity,
                "payment_method": payment_method
            })
            sale_id += 1

df_sales = pd.DataFrame(sales)
df_sales = df_sales.merge(df_products[["product_id", "price"]], on="product_id")
df_sales["total"] = df_sales["quantity"] * df_sales["price"]

# Save generated data
df_products.to_csv("products.csv", index=False)
df_customers.to_csv("customers.csv", index=False)
df_sales.to_csv("sales.csv", index=False)

print("✅ Files generated: products.csv, customers.csv, sales.csv with regional and payment method distributions.")


In [ ]:
df_products = pd.read_csv('products.csv')
df_customers = pd.read_csv('customers.csv')
df_sales = pd.read_csv('sales.csv')

In [ ]:
len(df_products)

In [ ]:
len(df_customers)

In [ ]:
len(df_sales)

In [ ]:
df_sales